In [ ]:
import talib
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import date
import bt

ticker = "aapl"
today = date.today().strftime("%Y-%m-%d")
stock_data = yf.download(ticker, start="2020-1-1", end=today)
stock_data.tail()
EMA_short = talib.EMA(stock_data['Close'], timeperiod=12).to_frame()
EMA_short = EMA_short.rename(columns={0: 'Close'})
EMA_long = talib.EMA(stock_data['Close'], timeperiod=50).to_frame()
EMA_long = EMA_long.rename(columns={0: 'Close'})
signal = EMA_long.copy()
signal[EMA_long.isnull()] = 0
signal[EMA_short > EMA_long] = 1
signal[EMA_short < EMA_long] = -1
signal[50:250]
transition = signal[signal['Close'].diff() != 0]
buy_signal = transition[transition['Close'] == 1]
sell_signal = transition[transition['Close'] == -1]

buy_index = buy_signal.index
buy_position = stock_data[stock_data.index.isin(buy_index)]
sell_index = sell_signal.index
sell_position = stock_data[stock_data.index.isin(sell_index)]

fig = go.Figure()
fig.add_trace(
        go.Candlestick(x=stock_data.index,
                        open=stock_data['Open'],
                        high=stock_data['High'],
                        low=stock_data['Low'],
                        close=stock_data['Close'],
                        name="Stock Prices"
                      )
)

fig.add_trace(
        go.Scatter(
            x=stock_data.index,
            y=EMA_long['Close'],
            name="EMA 50"
        )
)

fig.add_trace(
        go.Scatter(
            x=stock_data.index,
            y=EMA_short['Close'],
            name = "EMA 12"
        )
)

fig.add_trace(
        go.Scatter(
            x=buy_position.index,
            y=buy_position['Close'],
            name="Buy Signal",
            marker=dict(color="#511CFB", size=15),
            mode="markers",
            marker_symbol="triangle-up"
        )
)

fig.add_trace(
        go.Scatter(
            x=sell_position.index,
            y=sell_position['Close'],
            name="Sell Signal",
            marker=dict(color="#750086", size=15),
            mode="markers",
            marker_symbol="triangle-down"
        )
)

fig.update_layout(
    xaxis_rangeslider_visible=False,
    title="Daily Close (" + ticker + ") Prices",
    xaxis_title="Date",
    yaxis_title="Price (USD)"
)
fig.show()
%matplotlib inline

bt_strategy = bt.Strategy('EMA_crossover',
                            [   bt.algos.RunWeekly(),
                                bt.algos.WeighTarget(signal),
                                bt.algos.Rebalance()
                            ]
                         )

bt_backtest = bt.Backtest(bt_strategy, stock_data['Close'].to_frame())
bt_result = bt.run(bt_backtest)
bt_result.plot(title='Backtest result (Equity Progression)')
plt.show()
bt_result.plot_histograms(bins=50, freq = 'w')

bt_result.display()